# INF582 AXA Challenge

## Initialisation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Load main librairies
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import itertools
import random

## Load data

In [ ]:
# Load data
training_data = pd.read_csv("data/train_2011_2012.csv", sep=';', nrows=10000)
training_data.head()

## Clean data

In [ ]:
# Remove columns containing only one value
for name in training_data.columns:
    count = training_data[name].unique().size
    if count <= 1:
        training_data.drop(name, 1, inplace=True)